# Wind Evolution Analysis

This notebook analyzes the wind evolution system in our sailing environment. We'll explore:
1. How wind patterns evolve over time
2. The consistency of wind evolution with the same seed
3. The variability of wind evolution with different seeds
4. The different training and test scenarios

The analysis will help us understand and validate the wind system before implementing sailing agents.

## Setup
First, let's import the necessary modules and set up our environment.

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Add the src directory to the path
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('..'))

# Import the environment and evaluation modules
from src.env_sailing import SailingEnv
from src.evaluation import evaluate_agent, visualize_trajectory
from src.agents.agent_smart import AgentSmart
from src.agents.base_agent import BaseAgent

# Add this line with the other imports at the top of the notebook
from scenarios import get_scenario, SCENARIOS

## Static Agent
To focus on wind visualization without the complexity of agent movement, we'll create a `StaticAgent` that stays in place. This agent will help us observe how the wind evolves around a fixed point.

In [2]:
# Create static agent (needs to inherit from BaseAgent now)
class StaticAgent(BaseAgent):
    """Agent that stays in place to help visualize wind evolution."""
    
    def act(self, observation: np.ndarray) -> int:
        return 8  # The "stay in place" action

## Wind Evolution Parameters
We'll define parameters for both wind initialization and evolution. These parameters control:
- Base wind speed and direction
- Spatial patterns and variations
- Evolution characteristics (change probability, perturbations)
- Directional bias in evolution

In [3]:
# Define a scenario for wind evolution analysis
scenario = {
    'wind_init_params': {
        'base_speed': 4.0,
        'base_direction': (0.65, -0.75),  # More northern-oriented
        'pattern_scale': 32,         # Scale of the anticyclonic pattern
        'pattern_strength': 0.4,     # Wind direction variation
        'strength_variation': 0.5,   # Wind strength variation
        'noise': 0.1                 # Stochastic component
    },
    'wind_evol_params': {
        'wind_change_prob': 1,     # 10% chance of wind change each step
        'pattern_scale': 128,        # Scale of evolution pattern
        'perturbation_angle_amplitude': 0.2,  # Amplitude of wind direction changes
        'perturbation_strength_amplitude': 0.2,  # Amplitude of wind speed changes
        'wind_evolution_bias': (-1, 0.0),  # No bias by default
        'bias_strength': 0.2        # How strongly to apply any bias
    },
    'env_params': {
        'wind_grid_density': 25,    # Fewer arrows = clearer visualization
        'wind_arrow_scale': 80      # Larger value = smaller arrows
    }
}

## Wind Evolution Analysis
Let's analyze wind evolution with different seeds to understand:
1. The stochastic nature of wind changes
2. How different seeds produce different patterns
3. The impact of evolution parameters on wind behavior

In [4]:
# Create static agent instance
static_agent = StaticAgent()

# Evaluate with different seeds, with rendering enabled
print("Environment 1 (seed=40):")
results1 = evaluate_agent(
    agent=static_agent,
    scenario=scenario,
    seeds=40,
    max_horizon=100,
    verbose=False,
    render=True,
    full_trajectory=True  # Enable full trajectory for visualization
)
visualize_trajectory(results1, None, with_slider=True)

print("\nEnvironment 2 (seed=43):")
results2 = evaluate_agent(
    agent=static_agent,
    scenario=scenario,
    seeds=43,
    max_horizon=100,
    verbose=False,
    render=True,
    full_trajectory=True
)
visualize_trajectory(results2, None, with_slider=True)

print("\nEnvironment 3 (seed=44):")
results3 = evaluate_agent(
    agent=static_agent,
    scenario=scenario,
    seeds=44,
    max_horizon=100,
    verbose=False,
    render=True,
    full_trajectory=True
)
visualize_trajectory(results3, None, with_slider=True)

Environment 1 (seed=40):


/Users/t.rahier/code_project_v0/Sailing_project_v1/src/env_sailing.py:537: RuntimeWarning: invalid value encountered in divide
  direction_normalized = direction / np.linalg.norm(direction)


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…


Environment 2 (seed=43):


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…


Environment 3 (seed=44):


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…

## Training and Test Scenarios Visualization

Now that we understand how wind evolution works, let's examine the predefined scenarios that will be used for training and testing agents. We have:

1. **Training Scenarios**:
   - `training_1`: Clockwise rotating wind with eastward bias
   - `training_2`: Counter-clockwise rotation with westward bias
   - `training_3`: North-South oscillating wind pattern

2. **Test Scenarios**: (WILL BE KEPT PRIVATE TO THE STUDENTS)
   - `test`: Complex scenario combining elements from training scenarios

For each scenario, we'll use our `StaticAgent` to visualize:
- The initial wind pattern
- How the wind evolves over time
- The effect of the scenario's specific parameters (bias, strength, evolution probability)

This will help us understand what challenges each scenario presents to the agents and validate that they provide a good progression from training to testing.

In [6]:
# Visualize training scenarios with consistent seed
print("# Training Scenarios Comparison\n")
print("Comparing the three training scenarios with the same seed to understand their distinct characteristics.\n")

# Set evaluation parameters
eval_params = {
    'max_horizon': 100,
    'verbose': False,
    'render': True,
    'full_trajectory': True
}

# Set visualization parameters to add to scenarios
viz_params = {
    'env_params': {
        'wind_grid_density': 25,    # Fewer arrows = clearer visualization
        'wind_arrow_scale': 80      # Larger value = smaller arrows
    }
}

# Create static agent
static_agent = StaticAgent()

# Evaluate each training scenario
print("## Training Scenario 1: Clockwise Rotating Wind")
scenario1 = get_scenario('training_1')
print("\nKey characteristics:")
print(f"- Base wind: speed={scenario1['wind_init_params']['base_speed']:.1f}, "
      f"direction={scenario1['wind_init_params']['base_direction']}")
print(f"- Pattern strength: {scenario1['wind_init_params']['pattern_strength']:.2f}")
print(f"- Evolution probability: {scenario1['wind_evol_params']['wind_change_prob']:.2f}")
print(f"- Evolution bias: {scenario1['wind_evol_params']['wind_evolution_bias']}")
print(f"- Bias strength: {scenario1['wind_evol_params']['bias_strength']:.2f}")
print("\nThis scenario features a clockwise rotating wind pattern, challenging the agent to adapt to systematic directional changes.")

scenario1.update(viz_params)  # Add visualization parameters
results1 = evaluate_agent(
    agent=static_agent,
    scenario=scenario1,
    seeds=42,
    **eval_params
)
visualize_trajectory(results1, None, with_slider=True)

print("\n## Training Scenario 2: Counter-Clockwise Rotation")
scenario2 = get_scenario('training_2')
print("\nKey characteristics:")
print(f"- Base wind: speed={scenario2['wind_init_params']['base_speed']:.1f}, "
      f"direction={scenario2['wind_init_params']['base_direction']}")
print(f"- Pattern strength: {scenario2['wind_init_params']['pattern_strength']:.2f}")
print(f"- Evolution probability: {scenario2['wind_evol_params']['wind_change_prob']:.2f}")
print(f"- Evolution bias: {scenario2['wind_evol_params']['wind_evolution_bias']}")
print(f"- Bias strength: {scenario2['wind_evol_params']['bias_strength']:.2f}")
print("\nThis scenario presents a counter-clockwise rotating wind pattern, offering a different directional challenge from Scenario 1.")

scenario2.update(viz_params)
results2 = evaluate_agent(
    agent=static_agent,
    scenario=scenario2,
    seeds=42,
    **eval_params
)
visualize_trajectory(results2, None, with_slider=True)

print("\n## Training Scenario 3: Oscillating Wind")
scenario3 = get_scenario('training_3')
print("\nKey characteristics:")
print(f"- Base wind: speed={scenario3['wind_init_params']['base_speed']:.1f}, "
      f"direction={scenario3['wind_init_params']['base_direction']}")
print(f"- Pattern strength: {scenario3['wind_init_params']['pattern_strength']:.2f}")
print(f"- Evolution probability: {scenario3['wind_evol_params']['wind_change_prob']:.2f}")
print(f"- Evolution bias: {scenario3['wind_evol_params']['wind_evolution_bias']}")
print(f"- Bias strength: {scenario3['wind_evol_params']['bias_strength']:.2f}")
print("\nThis scenario features an oscillating wind pattern, where the wind direction periodically shifts back and forth.")

scenario3.update(viz_params)
results3 = evaluate_agent(
    agent=static_agent,
    scenario=scenario3,
    seeds=42,
    **eval_params
)
visualize_trajectory(results3, None, with_slider=True)

print("\n## Test Scenario: Complex Evolution")
scenario_test = get_scenario('test')
print("\nKey characteristics:")
print(f"- Base wind: speed={scenario_test['wind_init_params']['base_speed']:.1f}, "
      f"direction={scenario_test['wind_init_params']['base_direction']}")
print(f"- Pattern strength: {scenario_test['wind_init_params']['pattern_strength']:.2f}")
print(f"- Evolution probability: {scenario_test['wind_evol_params']['wind_change_prob']:.2f}")
print(f"- Evolution bias: {scenario_test['wind_evol_params']['wind_evolution_bias']}")
print(f"- Bias strength: {scenario_test['wind_evol_params']['bias_strength']:.2f}")
print("\nThis test scenario combines various wind patterns and evolution characteristics, testing the agent's ability to handle complex, unpredictable wind conditions.")

scenario_test.update(viz_params)
results_test = evaluate_agent(
    agent=static_agent,
    scenario=scenario_test,
    seeds=42,
    **eval_params
)
visualize_trajectory(results_test, None, with_slider=True)

# Training Scenarios Comparison

Comparing the three training scenarios with the same seed to understand their distinct characteristics.

## Training Scenario 1: Clockwise Rotating Wind

Key characteristics:
- Base wind: speed=4.0, direction=(-0.65, -0.75)
- Pattern strength: 0.30
- Evolution probability: 1.00
- Evolution bias: (1, 0.0)
- Bias strength: 0.20

This scenario features a clockwise rotating wind pattern, challenging the agent to adapt to systematic directional changes.


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…


## Training Scenario 2: Counter-Clockwise Rotation

Key characteristics:
- Base wind: speed=4.0, direction=(0.65, -0.75)
- Pattern strength: 0.40
- Evolution probability: 1.00
- Evolution bias: (-1, 0.0)
- Bias strength: 0.20

This scenario presents a counter-clockwise rotating wind pattern, offering a different directional challenge from Scenario 1.


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…


## Training Scenario 3: Oscillating Wind

Key characteristics:
- Base wind: speed=4.0, direction=(0.0, -1.0)
- Pattern strength: 0.30
- Evolution probability: 0.80
- Evolution bias: (0.0, 0.0)
- Bias strength: 0.00

This scenario features an oscillating wind pattern, where the wind direction periodically shifts back and forth.


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…


## Test Scenario: Complex Evolution

Key characteristics:
- Base wind: speed=4.0, direction=(-0.3, -0.95)
- Pattern strength: 0.35
- Evolution probability: 0.90
- Evolution bias: (0.5, 0.0)
- Bias strength: 0.15

This test scenario combines various wind patterns and evolution characteristics, testing the agent's ability to handle complex, unpredictable wind conditions.


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…